## <center><strong><span style= "color: #51fcc6">Notebook </span>Describtion

Hello There In this notebokk we are going to scrape the Full time and part time jobs data for the data related jobs like:<br>
Data analyst, ML dev, Data scientist, Data engineer etc ..<br><br>

**Important note:**
We will analyze some countries only for indeed becuase indeed needs to specify which country to look at.

## <center><strong>Importing <span style= "color: #48e0dc">Packeges

In [1]:
import time
import sqlite3
import warnings
import requests
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from itertools import count
from bs4 import BeautifulSoup
from selenium import webdriver
import matplotlib.pyplot as plt
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from IPython.display import set_matplotlib_formats
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

In [2]:
COLORS = ["#51fcc6", "#48e0dc", "#5cd3f7", "#4895e0", "#517afc"]

%matplotlib inline

NUMERICS = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
            'uint16', 'uint32', 'uint64']
MAX_PAGES = 300

set_matplotlib_formats('pdf', 'svg')
warnings.filterwarnings("ignore")

C:\Users\FreeComp\AppData\Local\Temp\ipykernel_19220\1856627373.py:9: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('pdf', 'svg')


## <center><strong>Setting up the<span style= "color: #5cd3f7"> web scrapers

In [3]:
data_jobs_titles = ["Data entry", "Data engineer",
                    "Data scientist", "Data analyst",
                    "ML devoloper"]

indeed_countries = ['uk', 'de', 'www'] # USA don't have to be spesfied in the URL so we will use 'www' instead
linkedin_countries = ['European Union', 'United States']

In [4]:
def scrape_page(url: str,
                ip_and_port: tuple[str]= None,
                retrieve_new_url= None) -> BeautifulSoup:

    if ip_and_port:
        chrome_options = Options()
        chrome_options.add_argument(f'--proxy-server={ip_and_port[0]}:{ip_and_port[1]}')
        
        webdriver_service = Service(GeckoDriverManager().install())
        driver = webdriver.Chrome(service= webdriver_service, options= chrome_options)

    else:
        driver = webdriver.Chrome()

    driver.get(url)
    
    wait = WebDriverWait(driver, 10)
    
    # Wait until the page is fully loaded
    time.sleep(3)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    html = driver.page_source

    if retrieve_new_url:

        new_url = driver.current_url
        driver.quit()

        return BeautifulSoup(html), new_url

    driver.quit()

    return BeautifulSoup(html)

In [5]:
def indeed_scraper(page: int,
                   country: str,
                   data_job_title: str) -> BeautifulSoup:

    page = (page - 1) * 10
    data_job_title = data_job_title.replace(' ', '+')

    # Pages in indeed.com are zero based indexed
    url: str = f'https://{country}.indeed.com/jobs?q={data_job_title}&sc=0kf%3Aattr%28DSQF7%29%3B&start=f{page}' + \
    '&pp=gQClAAAAAAAAAAAAAAACCXklWwCSAQIBK7YBBwDUbujiRGBahYW5TppjdUz7DjXn3aPZSbT47IoJ5LLbuzpYcXwZdzJ6rKHf' + \
    '6gPWFkXVGxKKGxW-JAKb8BFo_hZAkBd7trBBTY32J2CrOuA3V9dGD_bre-lArmi9DRYlcah6hvoRfsYUNYSoQwIa8VOMZMxvH-s2Dlh' + \
    'UPvUP-_Dz9ls4i-OLqVGpGh4AAA&vjk=b997bb0dddadea'

    soup = scrape_page(url)

    loaded_page = int(soup.find('button', 'css-ns2mzi e8ju0x51').text)
    available_pages = []
                       
    for button in soup.find_all('button', 'css-1qt7hdn e8ju0x50'):
        available_pages.append(int(button.text))

    print(url)
    print(loaded_page)
    print((page / 10) + 1)

    if ((page / 10) + 1 != loaded_page) or ((page / 10) + 1 not in available_pages):
        # We will use KeyError as standard to represent getting out of the max pages.
        raise KeyError

    return  soup

In [36]:
def is_element_present(driver):
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    try:
        driver.find_element(By.CLASS_NAME, 'loader loader--show')
        continue_ = True

    except:
        try:
            driver.find_element(By.CLASS_NAME, 'infinite-scroller__show-more-button infinite-scroller__show-more-button--visible')
            continue_ = True

        except:
            continue_ = False

    print(continue_)
    return  continue_

In [37]:
def linkedin_scraper(country: str, # This scraper scrapes all pages at once
                     data_job_title: str) -> BeautifulSoup:

    # I know that code logic is wrong but at least it's running completly fine.

    driver = webdriver.Chrome()
    url: str = f'https://www.linkedin.com/jobs/search/?keywords={data_job_title}&' + \
                f'location={country}&refresh=true&start=0&position=1&pageNum=0'

    driver.get(url)

    while True:
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            wait = WebDriverWait(driver, 10)
            wait.until(lambda driver: is_element_present(driver))
            print(is_element_present(driver))

        except:
            break

    soup = BeautifulSoup(driver.page_source)

    return  soup

In [126]:
indeed_scraper(2, 'uae', 'Data analyst')

https://uae.indeed.com/jobs?q=Data+analyst&sc=0kf%3Aattr%28DSQF7%29%3B&start=f10&pp=gQClAAAAAAAAAAAAAAACCXklWwCSAQIBK7YBBwDUbujiRGBahYW5TppjdUz7DjXn3aPZSbT47IoJ5LLbuzpYcXwZdzJ6rKHf6gPWFkXVGxKKGxW-JAKb8BFo_hZAkBd7trBBTY32J2CrOuA3V9dGD_bre-lArmi9DRYlcah6hvoRfsYUNYSoQwIa8VOMZMxvH-s2DlhUPvUP-_Dz9ls4i-OLqVGpGh4AAA&vjk=b997bb0dddadea
1
2.0


KeyError: 

In [38]:
linkedin_scraper('United States', 'Data scientist')

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


<html lang="en"><head>
<meta content="d_jobs_guest_search" name="pageKey"/>
<meta content="urlType=jserp_custom;emptyResult=false" name="linkedin:pageTag"/>
<meta content="en_US" name="locale"/>
<meta data-app-version="2.0.1312" data-browser-id="fb61d539-b636-45f2-834e-d15308c7cadd" data-call-tree-id="AAYAQ+7nl5G4Vjr407uxsA==" data-disable-jsbeacon-pagekey-suffix="false" data-enable-page-view-heartbeat-tracking="" data-member-id="0" data-multiproduct-name="jobs-guest-frontend" data-page-instance="urn:li:page:d_jobs_guest_search;XA1yhyb8QcazAPH6XcJdsA==" data-service-name="jobs-guest-frontend" id="config"/>
<link href="https://www.linkedin.com/jobs/data-scientist-jobs" rel="canonical"/>
<!-- --><!-- -->
<!-- -->
<!-- -->
<!-- -->
<!-- -->
<link href="https://static.licdn.com/aero-v1/sc/h/al2o9zrvru7aqj8e1x2rzsrca" rel="icon"/>
<script async="" charset="utf-8" id="utag_94" src="https://platform.linkedin.com/litms/vendor/google/gtag-cm-dv360-sa360.js?id=DC-9261636" type="text/javascript">